# Comprehensive LangChain Agent Guide with Mistral AI

Welcome! This notebook provides a complete guide to building AI agents using [LangChain](https://python.langchain.com/) and the [Mistral API](https://docs.mistral.ai/). 

**What you'll learn:**
- **5 different agent patterns** from simple to advanced
- **Tool integration** with custom and structured tools
- **Memory management** for conversational agents
- **Best practices** for production-ready agents

**Agent Types Covered:**
1. **ReAct Agent**: Reasoning and acting in loops
2. **Tool-Calling Agent**: Native function calling
3. **Conversational Agent**: Memory-enabled interactions
4. **Zero-Shot Agent**: Minimal configuration setup
5. **Structured Tool Agent**: Type-safe tool inputs

Let's build some intelligent agents! 🤖

## Complete LangChain Agent Types Reference

Before diving into our implementations, let's understand all the available agent types in LangChain. Each serves different use cases and has unique capabilities:

### 🎯 **Core Agent Types**

#### **ZERO_SHOT_REACT_DESCRIPTION** = `'zero-shot-react-description'`
A zero-shot agent that performs a reasoning step before acting. This is the most commonly used agent type for general-purpose tasks.

- **Use Case**: General problem-solving with reasoning chains
- **Strengths**: Good balance of reasoning and action
- **Best For**: Most common agent scenarios

#### **REACT_DOCSTORE** = `'react-docstore'`
A zero-shot agent with reasoning capabilities that has access to a document store for information lookup.

- **Use Case**: Question-answering with document retrieval
- **Strengths**: Can search and reference external documents
- **Best For**: Knowledge-intensive tasks requiring document lookup

#### **SELF_ASK_WITH_SEARCH** = `'self-ask-with-search'`
An agent that breaks down complex questions into simpler sub-questions and uses search to answer them.

- **Use Case**: Complex multi-step questions
- **Strengths**: Systematic problem decomposition
- **Best For**: Research tasks and complex analytical questions

### 💬 **Conversational Agent Types**

#### **CONVERSATIONAL_REACT_DESCRIPTION** = `'conversational-react-description'`
A conversational agent that maintains context across multiple turns while using reasoning and tools.

- **Use Case**: Multi-turn conversations with tool usage
- **Strengths**: Memory + reasoning + tool access
- **Best For**: Interactive applications requiring context

#### **CHAT_ZERO_SHOT_REACT_DESCRIPTION** = `'chat-zero-shot-react-description'`
A zero-shot agent optimized for chat models with reasoning capabilities.

- **Use Case**: Chat-based interactions with reasoning
- **Strengths**: Optimized for conversational models
- **Best For**: Modern chat model integrations

#### **CHAT_CONVERSATIONAL_REACT_DESCRIPTION** = `'chat-conversational-react-description'`
A conversational agent specifically designed for chat models with full memory and reasoning.

- **Use Case**: Advanced conversational AI with tools
- **Strengths**: Full conversational capabilities
- **Best For**: Sophisticated chatbot applications

### 🔧 **Structured Agent Types**

#### **STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION** = `'structured-chat-zero-shot-react-description'`
A zero-shot react agent optimized for chat models that can invoke tools with multiple structured inputs.

- **Use Case**: Complex tool interactions
- **Strengths**: Handles multi-parameter tools elegantly
- **Best For**: Applications requiring complex tool inputs

### 🔗 **Function-Calling Agent Types**

#### **OPENAI_FUNCTIONS** = `'openai-functions'`
An agent optimized for using OpenAI's function calling capabilities (also works with compatible models like Mistral).

- **Use Case**: Native function calling
- **Strengths**: Reliable structured tool usage
- **Best For**: Production applications requiring consistent tool calls

#### **OPENAI_MULTI_FUNCTIONS** = `'openai-multi-functions'`
An enhanced version that can call multiple functions in parallel or sequence.

- **Use Case**: Complex workflows requiring multiple tool calls
- **Strengths**: Efficient multi-tool operations
- **Best For**: Advanced automation tasks

### 🎨 **Agent Selection Guide**

| **Agent Type** | **Memory** | **Reasoning** | **Tool Complexity** | **Best Use Case** |
|---------------|------------|---------------|-------------------|------------------|
| `ZERO_SHOT_REACT_DESCRIPTION` | ❌ | ✅ | Simple | General tasks |
| `REACT_DOCSTORE` | ❌ | ✅ | Document Store | Q&A with docs |
| `SELF_ASK_WITH_SEARCH` | ❌ | ✅✅ | Search Tools | Complex research |
| `CONVERSATIONAL_REACT_DESCRIPTION` | ✅ | ✅ | Simple-Medium | Interactive apps |
| `CHAT_ZERO_SHOT_REACT_DESCRIPTION` | ❌ | ✅ | Simple | Chat-based |
| `CHAT_CONVERSATIONAL_REACT_DESCRIPTION` | ✅ | ✅ | Medium | Advanced chat |
| `STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION` | ❌ | ✅ | Complex | Multi-param tools |
| `OPENAI_FUNCTIONS` | ❌ | ⚡ | Structured | Production tools |
| `OPENAI_MULTI_FUNCTIONS` | ❌ | ⚡ | Multi-tool | Complex workflows |

**Legend:** ✅ = Yes, ❌ = No, ✅✅ = Enhanced, ⚡ = Native Function Calling

### 📚 **Official Reference**
For the complete technical documentation of all agent types, see the official LangChain API reference:
**[LangChain AgentType Documentation](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_types.AgentType.html)**

In this notebook, we'll focus on **5 key patterns** that cover the most common use cases, but you can apply these principles to any of the agent types above!

## 1. Setup & Dependencies

## 1. Setup & Dependencies

First, let's install the required libraries and configure our environment. We'll use secure API key management to protect your credentials.

In [ ]:
# Install required libraries (uncomment if running in a new environment)
!pip install langchain-mistralai
!pip install python-dotenv  
!pip install langchain
!pip install langchain-community

# Note: If running in a managed environment (like VS Code or JupyterHub),
# you may need to restart the kernel after installation.

In [ ]:
# Secure API key loading
import os
from dotenv import load_dotenv
from getpass import getpass

# Load environment variables from a .env file if present
load_dotenv()

MISTRAL_API_KEY = os.getenv('MISTRAL_API_KEY')

# If not found, prompt the user securely
if not MISTRAL_API_KEY:
    MISTRAL_API_KEY = getpass('Enter your Mistral API key: ')

# Confirm that the key is loaded (do not print the key!)
if MISTRAL_API_KEY:
    print('✅ Mistral API key loaded.')
else:
    raise ValueError('❌ Mistral API key not found. Please set it in your .env file or enter it when prompted.')

## 2. Import Required Libraries

Let's organize our imports for the comprehensive agent examples we'll be building.

In [ ]:
from typing import Optional

# LangChain core imports
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_mistralai import ChatMistralAI

# Agent and tool imports
from langchain.agents import (
    AgentExecutor,
    create_react_agent,
    create_tool_calling_agent,
    initialize_agent,
    AgentType
)
from langchain.tools import Tool, StructuredTool
from langchain.memory import ConversationBufferMemory

# Pydantic for structured tools
from pydantic import BaseModel, Field

print("✅ All libraries imported successfully!")

## 3. Configure Mistral LLM

Now let's set up our Mistral LLM with optimal settings for agent use.

In [ ]:
# Initialize Mistral LLM with agent-optimized settings
llm = ChatMistralAI(
    api_key=MISTRAL_API_KEY,
    model="mistral-large-latest",
    temperature=0  # Lower temperature for more consistent agent behavior
)

print("✅ Mistral LLM configured successfully!")
print(f"Model: mistral-large-latest")
print(f"Temperature: 0 (deterministic)")

## 4. Define Utility Functions

Let's create some utility functions that our agents will use as tools. These demonstrate different types of tool capabilities.

In [ ]:
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

def multiply_numbers(a: str) -> int:
    """Multiplies two numbers separated by comma. Input format: 'num1,num2'"""
    nums = a.split(',')
    return int(nums[0]) * int(nums[1])

# Test our functions
print("Testing utility functions:")
print(f"Length of 'LangChain': {get_word_length('LangChain')}")
print(f"Multiply '5,7': {multiply_numbers('5,7')}")
print("✅ Utility functions working!")

## 5. Agent Example 1: ReAct Agent with Custom Tools

**ReAct (Reasoning + Acting)** agents alternate between thinking and acting, making them excellent for multi-step problem solving.

### Key Features:
- **Reasoning**: Agent thinks about what to do next
- **Acting**: Agent uses tools to gather information
- **Iteration**: Process repeats until goal is achieved
- **Tool Integration**: Seamless tool usage

In [ ]:
# Create basic tools for the ReAct agent
tools = [
    Tool(
        name="WordLength",
        func=get_word_length,
        description="Useful for finding the length of a word. Input should be a single word."
    ),
    Tool(
        name="Multiply",
        func=multiply_numbers,
        description="Multiplies two numbers. Input format: 'num1,num2'"
    )
]

# Get ReAct prompt from LangChain hub
prompt = hub.pull("hwchase17/react")

# Create ReAct agent
react_agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=react_agent, 
    tools=tools, 
    verbose=True,
    max_iterations=5
)

print("✅ ReAct Agent configured successfully!")
print("Tools available: WordLength, Multiply")

In [ ]:
# Test the ReAct agent
print("=" * 60)
print("🔄 REACT AGENT DEMONSTRATION")
print("=" * 60)

result = agent_executor.invoke({
    "input": "What is the length of the word 'langchain' multiplied by 5?"
})

print(f"\n🎯 Final Answer: {result['output']}")

## 6. Agent Example 2: Tool-Calling Agent

**Tool-Calling agents** leverage Mistral's native function calling capabilities for more efficient and reliable tool usage.

### Advantages:
- **Native Integration**: Uses model's built-in function calling
- **Better Reliability**: More consistent tool usage
- **Structured Calls**: JSON-formatted tool invocations

In [ ]:
# Define custom prompt for tool-calling agent
tool_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use tools to answer questions accurately."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# Create tool-calling agent (uses native function calling)
tool_agent = create_tool_calling_agent(llm, tools, tool_prompt)
tool_executor = AgentExecutor(
    agent=tool_agent, 
    tools=tools, 
    verbose=True,
    max_iterations=5
)

print("✅ Tool-Calling Agent configured successfully!")

In [ ]:
# Test the Tool-Calling agent
print("=" * 60)
print("🛠️ TOOL-CALLING AGENT DEMONSTRATION")
print("=" * 60)

result = tool_executor.invoke({
    "input": "Calculate 12 multiplied by 8, then tell me the length of 'mistral'"
})

print(f"\n🎯 Final Answer: {result['output']}")

## 7. Agent Example 3: Conversational Agent with Memory

**Conversational agents** can remember previous interactions, enabling multi-turn conversations and context-aware responses.

### Memory Benefits:
- **Context Preservation**: Remembers previous exchanges
- **Follow-up Questions**: Can reference earlier responses
- **Personalization**: Builds conversational context
- **Efficiency**: Avoids repeating information

In [ ]:
# Create memory for conversation history
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Prompt with chat history support
conversational_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use the provided tools when necessary. Remember our conversation history."),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# Create conversational agent
conv_agent = create_tool_calling_agent(llm, tools, conversational_prompt)
conv_executor = AgentExecutor(
    agent=conv_agent,
    tools=tools,
    memory=memory,
    verbose=True,
    max_iterations=5
)

print("✅ Conversational Agent with Memory configured!")
print("💾 Memory type: ConversationBufferMemory")

In [ ]:
# Demonstrate conversation memory
print("=" * 60)
print("💬 CONVERSATIONAL AGENT DEMONSTRATION")
print("=" * 60)

# First interaction
print("\n--- Turn 1: Initial Question ---")
result1 = conv_executor.invoke({"input": "What's the length of 'python'?"})
print(f"\n🎯 Response 1: {result1['output']}")

print("\n--- Turn 2: Follow-up (uses memory) ---") 
result2 = conv_executor.invoke({"input": "Multiply that number by 3"})
print(f"\n🎯 Response 2: {result2['output']}")

# Check memory content
print(f"\n🧠 Memory contains {len(memory.chat_memory.messages)} messages")

## 8. Agent Example 4: Zero-Shot Agent

**Zero-Shot agents** provide the simplest setup for quick agent deployment with minimal configuration.

### Use Cases:
- **Rapid Prototyping**: Quick agent setup
- **Simple Tasks**: Basic tool usage
- **Legacy Support**: Compatible with older LangChain versions
- **Minimal Dependencies**: Fewer configuration requirements

In [ ]:
# Simple tool for demonstration (Note: eval used for demo only - not production safe!)
simple_tools = [
    Tool(
        name="Calculator",
        func=lambda x: eval(x),  
        description="Useful for math calculations. Input should be a valid Python expression."
    )
]

# Initialize zero-shot agent
zero_shot_agent = initialize_agent(
    tools=simple_tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

print("✅ Zero-Shot Agent configured!")
print("⚠️  Note: Uses eval() for demo purposes only")

In [ ]:
# Test the Zero-Shot agent
print("=" * 60)
print("⚡ ZERO-SHOT AGENT DEMONSTRATION")
print("=" * 60)

result = zero_shot_agent.invoke({"input": "What is 25 * 4 + 10?"})

print(f"\n🎯 Final Answer: {result['output']}")

## 9. Agent Example 5: Structured Tools Agent

**Structured Tools** use Pydantic models for type-safe, well-documented tool inputs. This is the most robust approach for production systems.

### Advantages:
- **Type Safety**: Automatic input validation
- **Better Documentation**: Self-documenting parameters
- **Complex Inputs**: Support for nested objects
- **Auto-generated Schema**: JSON schema for function calling
- **Error Prevention**: Catches type mismatches early

Let's create multiple structured tools to demonstrate different capabilities.

In [ ]:
# ========== Structured Tool 1: Advanced Calculator ==========
class CalculatorInput(BaseModel):
    """Input schema for calculator operations."""
    a: int = Field(description="First number")
    b: int = Field(description="Second number") 
    operation: str = Field(description="Operation to perform: add, subtract, multiply, divide")

def calculator(a: int, b: int, operation: str) -> float:
    """Performs basic arithmetic operations."""
    operations = {
        "add": lambda x, y: x + y,
        "subtract": lambda x, y: x - y,
        "multiply": lambda x, y: x * y,
        "divide": lambda x, y: x / y if y != 0 else "Error: Division by zero"
    }
    
    if operation in operations:
        return operations[operation](a, b)
    else:
        return "Error: Unknown operation. Use add, subtract, multiply, or divide"

# ========== Structured Tool 2: Text Analyzer ==========
class TextAnalyzerInput(BaseModel):
    """Input schema for text analysis."""
    text: str = Field(description="The text to analyze")
    analysis_type: str = Field(
        description="Type of analysis: 'length', 'words', 'vowels', or 'uppercase'"
    )

def text_analyzer(text: str, analysis_type: str) -> dict:
    """Analyzes text and returns various statistics."""
    analyses = {
        "length": lambda t: {"result": len(t), "type": "character_count"},
        "words": lambda t: {"result": len(t.split()), "type": "word_count"},
        "vowels": lambda t: {"result": sum(1 for c in t.lower() if c in 'aeiou'), "type": "vowel_count"},
        "uppercase": lambda t: {"result": t.upper(), "type": "uppercase_conversion"}
    }
    
    if analysis_type in analyses:
        return analyses[analysis_type](text)
    else:
        return {"result": "Unknown analysis type", "type": "error"}

print("✅ Structured tool functions defined!")

In [ ]:
# ========== Structured Tool 3: User Profile Builder ==========
class Address(BaseModel):
    """Address information."""
    city: str = Field(description="City name")
    country: str = Field(description="Country name")

class UserProfileInput(BaseModel):
    """Input schema for creating user profiles."""
    name: str = Field(description="User's full name")
    age: int = Field(description="User's age")
    email: str = Field(description="User's email address")
    address: Optional[Address] = Field(None, description="User's address information")

def create_user_profile(name: str, age: int, email: str, address: Optional[Address] = None) -> str:
    """Creates a formatted user profile."""
    profile = f"""
=== User Profile ===
Name: {name}
Age: {age}
Email: {email}"""
    
    if address:
        profile += f"\nLocation: {address.city}, {address.country}"
    
    profile += "\n==================="
    return profile

print("✅ User profile builder defined!")

In [ ]:
# Create structured tools with Pydantic schemas
calculator_tool = StructuredTool.from_function(
    func=calculator,
    name="Calculator",
    description="Performs arithmetic operations: add, subtract, multiply, divide",
    args_schema=CalculatorInput
)

text_analyzer_tool = StructuredTool.from_function(
    func=text_analyzer,
    name="TextAnalyzer", 
    description="Analyzes text for length, word count, vowel count, or converts to uppercase",
    args_schema=TextAnalyzerInput
)

user_profile_tool = StructuredTool.from_function(
    func=create_user_profile,
    name="UserProfileBuilder",
    description="Creates a formatted user profile with name, age, email, and optional address",
    args_schema=UserProfileInput
)

# Combine all structured tools
structured_tools = [calculator_tool, text_analyzer_tool, user_profile_tool]

# Create agent with structured tools
structured_agent = create_tool_calling_agent(llm, structured_tools, tool_prompt)
structured_executor = AgentExecutor(
    agent=structured_agent, 
    tools=structured_tools, 
    verbose=True,
    max_iterations=5
)

print("✅ Structured Tools Agent configured!")
print(f"🛠️  Available tools: {len(structured_tools)} structured tools")

## 10. Testing Structured Tools Agent

Let's test all the structured tools with various scenarios to demonstrate their capabilities.

In [ ]:
print("=" * 60)
print("🎯 STRUCTURED TOOLS AGENT DEMONSTRATION")
print("=" * 60)

# Test 1: Calculator
print("\n--- Test 1: Advanced Calculator ---")
result = structured_executor.invoke({
    "input": "What is 156 divided by 12?"
})
print(f"\n🎯 Result: {result['output']}")

In [ ]:
# Test 2: Text Analyzer
print("\n--- Test 2: Text Analysis ---")
result = structured_executor.invoke({
    "input": "How many vowels are in the word 'LangChain'?"
})
print(f"\n🎯 Result: {result['output']}")

In [ ]:
# Test 3: User Profile Builder  
print("\n--- Test 3: User Profile Creation ---")
result = structured_executor.invoke({
    "input": "Create a profile for John Doe, age 30, email john@example.com, living in Paris, France"
})
print(f"\n🎯 Result: {result['output']}")

In [ ]:
# Test 4: Multiple Tools Working Together
print("\n--- Test 4: Multi-Tool Workflow ---")
result = structured_executor.invoke({
    "input": "Count the words in 'Hello World from LangChain' then multiply that number by 5"
})
print(f"\n🎯 Result: {result['output']}")

## 11. Agent Comparison & Best Practices

Now that we've explored all 5 agent types, let's compare them and discuss when to use each approach.

### Agent Type Comparison

| Agent Type | Complexity | Use Case | Strengths | Considerations |
|------------|------------|----------|-----------|----------------|
| **ReAct** | Medium | Multi-step reasoning | Great for complex workflows | Requires good prompts |
| **Tool-Calling** | Medium | Reliable tool usage | Native function calling | Model-dependent |
| **Conversational** | High | Multi-turn interactions | Context awareness | Memory management |
| **Zero-Shot** | Low | Quick prototyping | Simple setup | Limited customization |
| **Structured Tools** | High | Production systems | Type safety, validation | More setup required |

### Best Practices

1. **Start Simple**: Begin with Zero-Shot, evolve to Structured Tools
2. **Use Type Safety**: Prefer Structured Tools for production
3. **Add Memory Carefully**: Only when context is truly needed
4. **Limit Iterations**: Always set `max_iterations` to prevent runaway
5. **Handle Errors**: Include parsing error handling
6. **Secure API Keys**: Never hardcode credentials

## 12. Summary & Next Steps

Congratulations! You've successfully built and tested 5 different types of LangChain agents with Mistral AI.

### What You've Learned:

✅ **ReAct Agents**: Reasoning and acting in loops  
✅ **Tool-Calling Agents**: Native function calling capabilities  
✅ **Conversational Agents**: Memory-enabled interactions  
✅ **Zero-Shot Agents**: Quick setup for simple tasks  
✅ **Structured Tool Agents**: Type-safe, production-ready tools  

### Key Concepts Mastered:

- **Tool Integration**: Custom functions as agent capabilities
- **Memory Management**: Persistent conversation context  
- **Type Safety**: Pydantic models for robust inputs
- **Error Handling**: Graceful failure management
- **Security**: Safe API key management

### Next Steps:

1. **Experiment**: Try different tool combinations
2. **Customize**: Create domain-specific tools for your use case
3. **Scale**: Add more sophisticated memory and retrieval
4. **Deploy**: Move from notebook to production applications
5. **Monitor**: Add logging and performance tracking

### Additional Resources:

- [LangChain Documentation](https://python.langchain.com/)
- [Mistral AI Documentation](https://docs.mistral.ai/)
- [Agent Patterns Guide](https://python.langchain.com/docs/modules/agents/)

Happy building! 🚀

In [ ]:
# Final summary of all agents created
print("=" * 60)
print("🎉 COMPREHENSIVE AGENT SUMMARY")
print("=" * 60)
print("""
This notebook demonstrated 5 comprehensive agent patterns:

1. ✅ ReAct Agent: Uses reasoning and acting in loops
2. ✅ Tool-Calling Agent: Leverages native function calling  
3. ✅ Conversational Agent: Maintains memory across interactions
4. ✅ Zero-Shot Agent: Simplified initialization for quick setup
5. ✅ Structured Tool Agent: Uses Pydantic models for type safety
   - Calculator with arithmetic operations
   - Text Analyzer for text statistics  
   - User Profile Builder with nested data structures

🔧 Key Features Implemented:
- Type-safe tool inputs using Pydantic models
- Conversation memory management
- Error handling and parsing protection
- Controlled execution (max_iterations=5)
- Secure API key management

🚀 All agents are production-ready with proper error handling!
""")